In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Flatten, Conv2D
from keras.layers import Input, Add, Dense, BatchNormalization, Flatten, Conv2D, MaxPooling2D
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from keras.callbacks import Callback
from keras.models import Model
from keras.preprocessing.image import array_to_img
import os
%matplotlib inline
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [3]:
baseline_model = Sequential()
# Adding Conv layer  
baseline_model.add(Conv2D(filters=32, kernel_size = (3,3), activation="relu", input_shape=(224, 224, 3)))
# Added max pooling layer
baseline_model.add(MaxPooling2D(pool_size=(2,2)))
# Added normalization layer
baseline_model.add(BatchNormalization())
baseline_model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
baseline_model.add(MaxPooling2D(pool_size=(2,2)))
baseline_model.add(BatchNormalization())
# Added flatten option
baseline_model.add(Flatten())
baseline_model.add(BatchNormalization())
baseline_model.add(Dense(128,activation="relu"))
# Adding dropout of 50%
baseline_model.add(tf.keras.layers.Dropout(0.5))
baseline_model.add(Dense(2, activation="softmax"))
    
baseline_model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 111, 111, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                        

2022-03-23 00:15:14.101611: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-23 00:15:14.513017: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4154 MB memory:  -> device: 0, name: Quadro P2000, pci bus id: 0000:65:00.0, compute capability: 6.1


In [4]:
folder = "./chest_xray"
train_path = folder + '/train'
model_histories = []
model_scores = []
models = []
split_coefs = [0.1,0.15,0.20,0.25,0.3,0.35]
for split_coef in split_coefs:
    print("Training with split: ",str(split_coef))
    model = baseline_model
    model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy']
    )

    data_generator = ImageDataGenerator(preprocessing_function=tf.image.per_image_standardization,featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  
            featurewise_std_normalization=False,  
            samplewise_std_normalization=False,
            zca_whitening=False,  
            rotation_range=10, 
            zoom_range = 0.1, 
            width_shift_range=0.1,
            height_shift_range=0.1,  
            horizontal_flip=False,  
            vertical_flip=False,
            validation_split = split_coef)

    batch_size= 16

    train_generator = data_generator.flow_from_directory(directory=train_path, target_size=(224,224), classes=["normal","pneumonia"], batch_size=batch_size,subset='training')
    validation_generator = data_generator.flow_from_directory(directory=train_path, target_size=(224,224), classes=["normal","pneumonia"], batch_size=batch_size,subset='validation')

    filepath = "pneumonia-detection-model-via-loss-4-" + str(split_coef) + ".h5"
    from keras.callbacks import ModelCheckpoint

    checkpoint = ModelCheckpoint(filepath=filepath, 
                                 monitor="val_loss",
                                 verbose=1, 
                                 save_best_only=True,
                                 mode="auto")

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=0, patience=3, verbose=0,
        mode='auto', baseline=None, restore_best_weights=True
    )

    history = model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.samples // batch_size,
        validation_data = validation_generator, 
        validation_steps = validation_generator.samples // batch_size,
        epochs = 20,
        verbose = 1,
        callbacks=[checkpoint,early_stopping]
    )
    
    best_model_loss = history.history['val_loss'][np.argmin(history.history['val_loss'])]
    model_scores.append(best_model_loss)
    models.append(filepath)


Training with split:  0.1
Found 2238 images belonging to 2 classes.
Found 248 images belonging to 2 classes.


/tmp/ipykernel_95326/4270682621.py:47: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20


2022-03-23 00:15:16.234292: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302


139/139 [==============================] - ETA: 0s - loss: 2.5977 - accuracy: 0.8546
Epoch 1: val_loss improved from inf to 10.06284, saving model to pneumonia-detection-model-via-loss-4-0.1.h5
139/139 [==============================] - 46s 322ms/step - loss: 2.5977 - accuracy: 0.8546 - val_loss: 10.0628 - val_accuracy: 0.4958
Epoch 2/20
139/139 [==============================] - ETA: 0s - loss: 2.4356 - accuracy: 0.8812
Epoch 2: val_loss did not improve from 10.06284
139/139 [==============================] - 43s 310ms/step - loss: 2.4356 - accuracy: 0.8812 - val_loss: 15.9036 - val_accuracy: 0.5083
Epoch 3/20
139/139 [==============================] - ETA: 0s - loss: 2.1262 - accuracy: 0.8965
Epoch 3: val_loss improved from 10.06284 to 6.15319, saving model to pneumonia-detection-model-via-loss-4-0.1.h5
139/139 [==============================] - 45s 321ms/step - loss: 2.1262 - accuracy: 0.8965 - val_loss: 6.1532 - val_accuracy: 0.6792
Epoch 4/20
139/139 [=============================

In [5]:
print(np.argmin(model_scores))
split_coefs = [0.1,0.15,0.20,0.25,0.3,0.35]
best_split = split_coefs[np.argmin(model_scores)]
best_model = models[np.argmin(model_scores)]
print("Best model is: ", models[np.argmin(model_scores)])

2
Best model is:  pneumonia-detection-model-via-loss-4-0.2.h5


In [6]:
print(model_scores)

[1.4032646417617798, 1.227704644203186, 0.4561465084552765, 0.7760316729545593, 0.5028775334358215, 0.5902078747749329]
